In [1]:
import numpy as np
import random
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import h5py
import random
import pickle

/home/steich/NF_Prize_Data/nf_prize_env/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
user_emb_dict = {}
with open('../data/user_emb_bow_all_unweighted.tsv', 'r') as iFile:
    for line in iFile:
        line = line.replace('\n', '')
        line = line.split('\t')
        user_emb_dict[int(line[0])] = np.array([float(i) for i in line[1:]]) 

In [3]:
rating_weights = {1:-1, 2:-1, 3:1, 4:1, 5:1}

In [4]:
hf = h5py.File('../data/Full_NF.h5', 'r')
dset = hf["dataset_1"]
dset = dset[:]

In [5]:
movie_dict = {}
for row in dset:
    if row[0] not in movie_dict.keys():
        movie_dict[row[0]] = [(row[1], row[2])]
    else:
        movie_dict[row[0]].append((row[1], row[2]))

In [6]:
movie_names_dict = {}
with open('/home/steich/NF_Prize_Data/data/movie_titles.csv', 'r', encoding="ISO-8859-1") as iFile:
    for line in iFile:
        line = line.replace('\n', '')
        line = line.split(',')
        movie_names_dict[int(line[0])] = line[2]

In [7]:
with open('../data/name_one_hot_dict.pickle', 'rb') as handle:
    name_one_hot = pickle.load(handle)

In [ ]:
movie_embeddings = []
movie_names = []
for movie in movie_dict:
    users = [(name_one_hot[user[0]], user[1]) for user in movie_dict[movie]]
    weighted_user_embs = [user_emb_dict[user[0]] * rating_weights[user[1]] for user in users]
    movie_embeddings.append(np.mean(weighted_user_embs, axis=0))
    movie_names.append(movie_names_dict[movie])

In [ ]:
m_tsne = TSNE(perplexity=30, n_components=2, init='pca', n_iter=5000, method='exact')
n = len(movie_embeddings)
sample = random.sample(range(n), 500)
low_dim_embs = tsne.fit_transform([movie_embeddings[i] for i in sample])
labels = [movie_names[i] for i in sample]
plt.figure(figsize=(18,18))
for i, label in enumerate(labels):
    x, y = low_dim_embs[i, :]
    plt.scatter(x,y)
    plt.annotate(label,
                    xy=(x, y),
                    xytext=(5,2),
                    textcoords='offset points',
                    ha='right',
                    va='bottom')
plt.savefig("/home/steich/NF_Prize_Data/data/tsne_movie_embeddings3.png")
plt.show()